In [169]:
datapath = "../../data/"
tmppath = "../../tmp/02/"
outpath = "./../output/"
settingpath = "./../setting/"

import warnings
warnings.simplefilter('ignore')

In [170]:
import torch
import torchtext

print(torch.__version__)  # 1.3.1
print(torchtext.__version__)  # 0.5.0

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
# device = torch.device("cpu")
print(device)

1.13.1+cu117
0.14.1
cuda:0


# 前処理

## データ読み込み

In [171]:
import numpy as np
import pandas as pd

def read_text_file(path):
    with open(path, mode="r") as f:
        result = f.read().splitlines()
    return result

def read_label_file(path):
    result = np.loadtxt(path, dtype='int64')
    return result

def create_df(textpath, lablpath):
    result = pd.DataFrame({'text': read_text_file(textpath),
                           'label': read_label_file(lablpath)})
    return result

def create_testdf(textpath):
    result = pd.DataFrame({'text': read_text_file(textpath)})
    result["label"] = [0]*len(result)
    return result
    
train_df_origin = create_df(datapath + "text.train.txt", datapath + "label.train.txt")
dev_df_origin = create_df(datapath + "text.dev.txt", datapath + "label.dev.txt")
test_df_origin = create_testdf(datapath + "text.test.txt")

In [172]:
print(train_df_origin.head())

                                                text  label
0                     ぼけっとしてたらこんな時間。チャリあるから食べにでたいのに…      0
1  今日の月も白くて明るい。昨日より雲が少なくてキレイな〜 と立ち止まる帰り道。チャリなし生活も...      1
2                 早寝するつもりが飲み物がなくなりコンビニへ。ん、今日、風が涼しいな。      0
3                                           眠い、眠れない。      0
4    ただいま〜 って新体操してるやん!外食する気満々で家に何もないのに!テレビから離れられない…!      0


In [173]:
print(test_df_origin.head())

                                          text  label
0        ハッピーセット、またプラレールが始まるからしばらくマックばかりになりそう。      0
1               今日は天気が良いので外出がきもちよいです。秋晴れ良いですね。      0
2  あぁ〜そうなんだ。。。 やっぱライブでは芸人みんなわちゃわちゃしてるとこが見たかったな      0
3                               踊り場110話まできたぞこら      0
4                         カウコン行かれる方、楽しんで下さい〜！！      0


## テキストクリーニング

In [241]:
import collections
from sudachipy import tokenizer
from sudachipy import dictionary

tokenizer_obj = dictionary.Dictionary(dict="full").create()
mode = tokenizer.Tokenizer.SplitMode.C

clear_part_of_speech_list = [["助詞", "助動詞"],["数詞"]]

with open(tmppath + "stopwords.txt") as f:
    stopword_list = f.read().splitlines()

# 出現頻度が少ない単語をstopwordとする
def stopwords_occur(textlist, threshold):
    morphemelist = [tokenizer_obj.tokenize(text, mode) for text in textlist]
    words = []
    for morpheme in morphemelist:
        for word in morpheme:
            words.append(word.normalized_form())
    dic = collections.Counter(words)
    dic = {key:value for key, value in dic.items() if value<= threshold}
    return list(dic.keys())

stopwords_occur = stopwords_occur(train_df_origin["text"], 2)

stopword_list = []

stopword_list.extend(stopwords_occur)

print(stopword_list[:100])

['新体操', '表情筋', 'シャレード', 'モテキ', '小宮山', '夏樹', 'バッティングセンター', 'COWCOW', 'BIGBANG', '笑む', '立石', '生半可', '達者', 'たんと', 'しい', '外反母趾', '昔夢', 'チョコケーキ', '天地明察', '不確か', '連れ去る', '微か', '絡み付く', '投げ掛ける', '茶碗蒸し', 'シナモンロール', '́з', '2700', 'wy', '愛と誠', '在り来り', 'ミランダ', 'カー', 'お子様', '岩', '整骨院', 'テーピング', '丸腰', '整体', '釣れる', '自己否定', '自己肯定', '鬼太郎', '彦', '門', '豚骨', '茄子', '田楽', 'パトラッシュ', 'エビマヨ', '煮', 'ふじ', 'ピール', 'なか卯', 'ジャーナリスト', '祈り', 'さめる', 'メルヘン', '齧り付く', '同志', '求', '本が好き', '立ち読み', '流し読み', 'standardbookstore', 'ノーベル賞', '山中', '育む', '西田辺', '割り箸', 'ニーハイ', '歯痒い', '信託', '別口', '著作', '田中里奈', 'たなか', 'りな', 'ティーナカリーナ', 'キョンキョン', '悪の教典', 'ニモ', 'ケズ', '出家', 'がぶり', 'カブレ', 'たらふく', 'ホームパーティー', '祭りのあと', 'BARBEE BOYS', '差し込む', 'はだける', '生田斗真', 'てれび戦士', '精神年齢', '体年齢', '串カツ', '有耶無耶', '怪しむ', '挙式']


In [242]:
def text_cleaning(text, mode, clear_part_of_speech_list, stopword_list):
    words = []
    for word in tokenizer_obj.tokenize(text, mode):
        if word.part_of_speech()[0] not in clear_part_of_speech_list[0] and word.part_of_speech()[1] not in clear_part_of_speech_list[1] and word.normalized_form() not in stopword_list:
            words.append(word.normalized_form())
    return " ".join(words)

def df_cleaning(df):
    result_df = df.copy()
    result_df['text'] = df['text'].map(lambda x: text_cleaning(x, mode, clear_part_of_speech_list, stopword_list))
    return result_df

train_df = df_cleaning(train_df_origin)
dev_df = df_cleaning(dev_df_origin)
test_df = df_cleaning(test_df_origin)

In [243]:
print(train_df.head())

                                                text  label
0               ぼけっと 為る こんな 時間 。 ちゃり 有る 食べる 出る . . .      0
1  今日 月 白い 明るい 。 昨日 雲 少ない 奇麗   立ち止まる 帰り道 。 ちゃり なし...      1
2        早寝 為る 積もり 飲み物 なくなる コンビニ 。 んっ 、 今日 、 風 涼しい 。      0
3                                          眠い 、 眠る 。      0
4    只今 〜   為る ! 外食 為る 気 満々 家 何 無い ! テレビ 離れる . . . !      0


In [244]:
print(test_df.head())

                                  text  label
0               、 又 始まる 暫く マック 成る そう 。      0
1        今日 天気 良い 外出 気持ち 良い 。 秋晴れ 良い 。      0
2  あー そう 。 。 。   矢張り ライブ 芸人 皆 為る とこ 見る      0
3                          踊り場 話 来る 此れ      0
4            カウコン 行く 方 、 楽しむ 下さる 〜 ! !      0


## 辞書作成

In [245]:
from collections import Counter
from torchtext.vocab import vocab

counter = Counter()
for text in train_df["text"]:
    counter.update(text.split())

voc = vocab(counter, specials=(['<unk>', '<pad>']))
voc.set_default_index(voc['<unk>'])

print(voc.get_itos()[:30]) 

['<unk>', '<pad>', 'ぼけっと', '為る', 'こんな', '時間', '。', 'ちゃり', '有る', '食べる', '出る', '.', '今日', '月', '白い', '明るい', '昨日', '雲', '少ない', '奇麗', '立ち止まる', '帰り道', 'なし', '生活', '悪い', '無い', '早寝', '積もり', '飲み物', 'なくなる']


## Dataloader

In [246]:
import torchtext.transforms as T

text_transform = T.Sequential(
    T.VocabTransform(voc),
    T.ToTensor(padding_value=voc['<pad>'])
)

def collate_batch(batch):
    texts = text_transform([text.split() for (text, label) in batch])
    # texts = torch.t(texts)
    labels = torch.tensor([label+2 for (text, label) in batch], dtype=torch.long)
    return texts, labels

In [276]:
from torch.utils.data import DataLoader

batch_size = 512

train_loader = DataLoader(train_df.values, batch_size=batch_size, shuffle=True, collate_fn=collate_batch, num_workers=4)
dev_loader = DataLoader(dev_df.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch, num_workers=4)
test_loader = DataLoader(test_df.values, batch_size=batch_size, shuffle=False, collate_fn=collate_batch, num_workers=4)

In [277]:
batch = next(iter(train_loader))
x, t = batch
print(t.shape)
print(x.shape)

batch = next(iter(test_loader))
x, t = batch
print(t.shape)
print(x.shape)

torch.Size([512])
torch.Size([512, 54])
torch.Size([512])
torch.Size([512, 60])


# モデル定義

In [278]:
EMBEDDING_DIM = 200 # 単語の埋め込み次元数
LSTM_DIM = 128 # LSTMの隠れ層の次元数
VOCAB_SIZE =len(voc) # 全単語数
TAG_SIZE = 5 # ネットワークの最後のサイズ
DA = 64 # AttentionをNeural Networkで計算する際の重み行列のサイズ
R = 3 # Attentionを３層重ねて見る

In [279]:
import torch.nn as nn
import torch.nn.functional as F

class BiLSTMEncoder(nn.Module):
    def __init__(self, embedding_dim, lstm_dim, vocab_size):
        super(BiLSTMEncoder, self).__init__()
        self.lstm_dim = lstm_dim
        self.word_embeddings = nn.Embedding(vocab_size, embedding_dim)

        # 単語ベクトルを誤差逆伝播で更新させないためにrequires_gradをFalseに設定する
        # self.word_embeddings.requires_grad_ = False

        # bidirectional=Trueで双方向のLSTMを利用
        self.bilstm = nn.LSTM(embedding_dim, lstm_dim, batch_first=True, bidirectional=True)

    def forward(self, text):
        embeds = self.word_embeddings(text)

        # 各隠れ層のベクトルがほしいので第１戻り値を受け取る
        out, _ = self.bilstm(embeds)

        # 前方向と後ろ方向の各隠れ層のベクトルを結合したままの状態で返す
        return out

In [280]:
class SelfAttention(nn.Module):
    def __init__(self, lstm_dim, da, r):
        super(SelfAttention, self).__init__()
        self.lstm_dim = lstm_dim
        self.da = da
        self.r = r
        self.main = nn.Sequential(
            # Bidirectionalなので各隠れ層のベクトルの次元は２倍のサイズ
            nn.Linear(lstm_dim * 2, da), 
            nn.Tanh(),
            nn.Linear(da, r)
        )
    def forward(self, out):
        return F.softmax(self.main(out), dim=1)

In [281]:
class SelfAttentionClassifier(nn.Module):
    def __init__(self, lstm_dim, da, r, tagset_size):
        super(SelfAttentionClassifier, self).__init__()
        self.lstm_dim = lstm_dim
        self.r = r
        self.attn = SelfAttention(lstm_dim, da, r)
        self.main = nn.Linear(lstm_dim * 6, tagset_size)

    def forward(self, out):
        attention_weight = self.attn(out)
        m1 = (out * attention_weight[:,:,0].unsqueeze(2)).sum(dim=1)
        m2 = (out * attention_weight[:,:,1].unsqueeze(2)).sum(dim=1)
        m3 = (out * attention_weight[:,:,2].unsqueeze(2)).sum(dim=1)
        feats = torch.cat([m1, m2, m3], dim=1)
        return F.log_softmax(self.main(feats)), attention_weight

## 学習

In [282]:
import torch.optim as optim
from itertools import chain

encoder = BiLSTMEncoder(EMBEDDING_DIM, LSTM_DIM, VOCAB_SIZE).to(device)
classifier = SelfAttentionClassifier(LSTM_DIM, DA, R, TAG_SIZE).to(device)
loss_function = nn.NLLLoss()

# 複数のモデルを from itertools import chain で囲えばoptimizerをまとめて1つにできる
optimizer = optim.Adam(chain(encoder.parameters(), classifier.parameters()), lr=0.0001)

In [283]:
losses = []

for epoch in range(20):
    all_loss = 0

    for idx, batch in enumerate(train_loader):
        batch_loss = 0
        encoder.zero_grad()
        classifier.zero_grad()

        text_tensor, label_tensor = batch
        out = encoder(text_tensor.to(device))
        score, attn = classifier(out)
        batch_loss = loss_function(score, label_tensor.to(device))
        batch_loss.backward()
        optimizer.step()
        all_loss = batch_loss.item()
        
        # valloss
        if idx == 0:
            with torch.no_grad():
                for batch in dev_loader:
                    text_tensor, label_tensor = batch
                    out = encoder(text_tensor.to(device))
                    score, _ = classifier(out)
                    val_loss = loss_function(score, label_tensor.to(device)).item()
            
    print("epoch", epoch, "\t" , "loss", all_loss, "\t", "val loss", val_loss)

epoch 0 	 loss 1.5526013374328613 	 val loss 1.6318556070327759
epoch 1 	 loss 1.5020326375961304 	 val loss 1.552908182144165
epoch 2 	 loss 1.5608993768692017 	 val loss 1.570878028869629
epoch 3 	 loss 1.5135682821273804 	 val loss 1.5792490243911743
epoch 4 	 loss 1.4873039722442627 	 val loss 1.5867362022399902
epoch 5 	 loss 1.511178970336914 	 val loss 1.6014349460601807
epoch 6 	 loss 1.5230742692947388 	 val loss 1.5836420059204102
epoch 7 	 loss 1.4571787118911743 	 val loss 1.5693082809448242
epoch 8 	 loss 1.4610240459442139 	 val loss 1.599522590637207
epoch 9 	 loss 1.4555978775024414 	 val loss 1.5806852579116821
epoch 10 	 loss 1.387213110923767 	 val loss 1.6015608310699463
epoch 11 	 loss 1.356120228767395 	 val loss 1.6132428646087646
epoch 12 	 loss 1.3693211078643799 	 val loss 1.6226720809936523
epoch 13 	 loss 1.4051826000213623 	 val loss 1.6628316640853882
epoch 14 	 loss 1.3321012258529663 	 val loss 1.6423391103744507
epoch 15 	 loss 1.3431711196899414 	 val 

# 評価

In [284]:
from sklearn.metrics import classification_report

answer = []
prediction = []
with torch.no_grad():
    for batch in dev_loader:
        text_tensor, label_tensor = batch

        out = encoder(text_tensor.to(device))
        score, _ = classifier(out)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())
        answer += list(label_tensor.cpu().numpy())
print(classification_report(prediction, answer))

              precision    recall  f1-score   support

           0       0.12      0.22      0.16       169
           1       0.36      0.23      0.28       649
           2       0.53      0.30      0.38      1161
           3       0.27      0.51      0.36       447
           4       0.10      0.39      0.16        74

    accuracy                           0.32      2500
   macro avg       0.28      0.33      0.27      2500
weighted avg       0.40      0.32      0.33      2500



# 提出用

In [285]:
prediction = []
test_result_df = test_df.copy()

with torch.no_grad():
    for batch in test_loader:
        text_tensor, _ = batch

        out = encoder(text_tensor.to(device))
        score, _ = classifier(out)
        _, pred = torch.max(score, 1)

        prediction += list(pred.cpu().numpy())

test_result_df["label"] = [pred-2 for pred in prediction]
print(test_result_df.head())

                                  text  label
0               、 又 始まる 暫く マック 成る そう 。      0
1        今日 天気 良い 外出 気持ち 良い 。 秋晴れ 良い 。      1
2  あー そう 。 。 。   矢張り ライブ 芸人 皆 為る とこ 見る      0
3                          踊り場 話 来る 此れ     -1
4            カウコン 行く 方 、 楽しむ 下さる 〜 ! !      1


In [286]:
with open(outpath + "test.txt","w") as f:
    f.writelines("\n".join([str(pred-2) for pred in prediction]))